# Missing Data
We'll refer to missing data in general as null, NaN, NaT or NA values.

## Missing Data
* https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html

In [2]:
import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

### NaN: Missing numerical data

In [6]:
data = np.array([1, np.nan, 3, 4]) 
data.dtype

dtype('float64')

In [7]:
1 + np.nan

nan

In [8]:
0 * np.nan

nan

In [9]:
data.sum(), data.min(), data.max()

(nan, nan, nan)

In [10]:
np.nansum(data), np.nanmin(data), np.nanmax(data)

(8.0, 1.0, 4.0)

### NaN and None in Pandas

In [14]:
# Pandas al detectar valores númericos convierte el valor "None" a NaN
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

In [15]:
# Pandas al detectar valores distintos manitene el valor "None"
pd.Series([1, np.nan, 'hello', None])

0        1
1      NaN
2    hello
3     None
dtype: object

### Operating on NULL values

In [17]:
#isnull() and notnull() will return a Boolean mask over the data
data = pd.Series([1, np.nan, 'hello', None])
data.isnull() 

0    False
1     True
2    False
3     True
dtype: bool

In [19]:
#dropna() will drop all rows in which any null value is present.
data[data.notnull()]
data.dropna() 

0        1
2    hello
dtype: object

In [20]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [21]:
df.dropna()

,0,1,2
1,2.0,3.0,5


In [23]:
df.dropna(axis='columns')
df.dropna(axis=1)

,2
0,2
1,5
2,6


In [24]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [27]:
# "how" y " "thresh" permite eliminar filas o columnas con todos los valores NA, o la mayoría de los valores NA.
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [28]:
# thresh parameter lets you specify a minimum number of non-null values for the row/column to be kept
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


### Filling null values

In [33]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data = pd.Series([1, np.nan, 2, None, 3], index=['a', 'b', 'c', 'd', 'e'])
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [30]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [31]:
data.fillna("missing")

a          1
b    missing
c          2
d    missing
e          3
dtype: object

In [34]:
# forward-fill
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [35]:
# back-fill
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

The following table lists Python operators and their equivalent Pandas object methods:

Python Operator | Pandas Method(s)
--------------- | ---------------
+ | add()
- | sub(), subtract()
* | mul(), multiply()
/ | truediv(), div(), divide()
// | floordiv()
% | mod()
** | pow()


### UFuncs (NumPy Universal Functions): Index Alignment

#### Index alignment in Series

In [13]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
A + B

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

In [10]:
A.index | B.index

Int64Index([0, 1, 2, 3], dtype='int64')

In [12]:
A.add(B, fill_value=0)

0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64

#### Index alignment in DataFrame

In [110]:
df = pd.DataFrame(
    np.array([np.nan, 4, 6, np.nan]),
    columns=['A']
)
df

,A
0,NaN
1,4.0
2,6.0
3,NaN


In [111]:
df.fillna(0)

,A
0,0.0
1,4.0
2,6.0
3,0.0


In [112]:
df.fillna("missing")

,A
0,missing
1,4
2,6
3,missing


In [113]:
df.fillna(df.mean())

,A
0,5.0
1,4.0
2,6.0
3,5.0


In [135]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
A = pd.DataFrame(A)
B = pd.DataFrame(B)
A + B

,0
0,NaN
1,5.0
2,9.0
3,NaN


In [123]:
# Fill specific value
df['Sum'] = A.add(B, fill_value=0);df[['Sum']]
df = pd.concat([A, B], axis=1).fillna(0);df['Sum'] = df.sum(axis=1);df[['Sum']]

,Sum
0,2.0
1,5.0
2,9.0
3,5.0


In [26]:
# Fill with a function
fill = A.stack().mean()
A.add(B, fill_value=fill)

,0
0,6.0
1,5.0
2,9.0
3,9.0


In [127]:
df = pd.concat([A, B], axis=1).fillna(0)
df.columns = ['$a', '$b']
df - df.iloc[0]

,$a,$b
0,0.0,0.0
1,2.0,1.0
2,4.0,3.0
3,-2.0,5.0


In [146]:
# df = pd.concat([A, B], axis=1)
# df.subtract(df['R'], axis=0)
pd.concat([A, B], axis=1)
df.columns = ['$a', '$b']
df
df.subtract(df['$a'], axis=0)

,$a,$b
0,0.0,NaN
1,0.0,-3.0
2,0.0,-3.0
3,NaN,NaN


In [34]:
A.stack()

0  0    2
1  0    4
2  0    6
dtype: int64